In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
import warnings

In [2]:
#importing csv files
nations_one = pd.read_csv("nations_league_1.csv", index_col=0)
nations_two = pd.read_csv("nations_league_2.csv", index_col=0)
world_cup = pd.read_csv("world_cup.csv", index_col=0)
euro_qual = pd.read_csv("euro_qual.csv", index_col=0)
euro_2022 = pd.read_csv("euro_2022.csv", index_col=0)

In [3]:
# Function to replace abbreviations with full country names
def remove_abbreviation(opponent):
    return opponent.split(' ', 1)[1]

In [4]:
#combing all df into one combined df, cleaning up data 
combined = pd.concat([nations_one,nations_two,world_cup,euro_qual,euro_2022])
combined['Opponent'] = combined['Opponent'].apply(remove_abbreviation)
combined = combined[combined['Comp'] != 'Friendlies (M)']
combined.to_csv("matches.csv")

In [5]:
# Function to adjust rows where the match went to overtime and winner was determined by penalty shoot-out
def adjust_result(row):
    gf = row['GF']
    ga = row['GA']
    
    # Check if there are parentheses in both GF and GA
    if re.search(r'\(\d+\)', gf) and re.search(r'\(\d+\)', ga):
        # Extract the numbers inside the parentheses
        gf_shootout = int(re.search(r'\((\d+)\)', gf).group(1))
        ga_shootout = int(re.search(r'\((\d+)\)', ga).group(1))
        
        # Adjust the result based on shootout scores
        if gf_shootout > ga_shootout:
            return 'W'
        elif gf_shootout < ga_shootout:
            return 'L'
        else:
            return row['Result']  # In case it's still a draw (unlikely scenario)
    else:
        return row['Result']

In [6]:
combined['GF'] = combined['GF'].astype(str)
combined['GA'] = combined['GA'].astype(str)
combined['Result'] = combined.apply(adjust_result, axis=1)

In [7]:
# Function to create weighted average for goals for and goals against for matches where winner was determined by penalty shootout
def adjust_goals(goals):
    # Check if there are parentheses indicating shootout scores
    if re.search(r'\(\d+\)', goals):
        # Extract the regular and shootout goals
        regular_goals = int(re.search(r'^\d+', goals).group())
        shootout_goals = int(re.search(r'\((\d+)\)', goals).group(1))
        # Calculate the adjusted goals (average or weighted average)
        adjusted_goals = (regular_goals + shootout_goals) / 2
        return adjusted_goals
    else:
        # Return the regular goals if no shootout score exists
        return float(goals)


In [8]:
combined['GF'] = combined['GF'].apply(adjust_goals)
combined['GA'] = combined['GA'].apply(adjust_goals)

In [9]:
combined.shape

(1077, 26)

In [10]:
combined.columns = combined.columns.str.lower()
combined = combined.sort_values(by="date")

venue_mapping = {'Home': 1, 'Away': 2, 'Neutral': 3}
combined['venue_num'] = combined['venue'].map(venue_mapping).astype(int)

result_mapping = {'L': 1, 'D': 2, 'W': 3}
combined = combined.dropna(subset=['result'])
combined['target'] = combined['result'].map(result_mapping).astype(int)
combined = combined.dropna(subset=['saves'])
combined['saves'] = combined['saves'].astype(int)
combined = combined.drop(columns=['xg', 'xga'])

In [11]:
#function to create rolling avg for stats
def rolling_avg(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed = 'left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset = new_cols)
    return group

In [12]:
cols = ["gf", "ga", "sh", "sot", "pk", "pkatt", "saves", "cs"]
new_cols = [f"{c}_rolling" for c in cols]

combined_rolling = combined.groupby('nation').apply(lambda x: rolling_avg(x, cols, new_cols))
combined_rolling = combined_rolling.droplevel('nation')
combined_rolling = combined_rolling.sort_values(by="date")

C:\Users\Sri\AppData\Local\Temp\ipykernel_35156\3335518567.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_rolling = combined.groupby('nation').apply(lambda x: rolling_avg(x, cols, new_cols))


In [13]:
# Define the features to use for each team
features = ['gf_rolling', 'ga_rolling', 'sh_rolling', 'sot_rolling', 'pk_rolling', 'pkatt_rolling', 'saves_rolling', 'cs_rolling']

# Group by 'nation' and calculate the mean of the features
team_stats_df = combined_rolling.groupby('nation')[features].mean()

# Convert the DataFrame to a dictionary where each key is a nation and each value is a list of features
team_stats = team_stats_df.apply(lambda row: row.tolist(), axis=1).to_dict()

In [14]:
combined_rolling['target'] = combined_rolling['result']
combined_rolling["venue_code"] = combined_rolling ["venue"].astype("category").cat.codes
combined_rolling["opp_code"] = combined_rolling["opponent"].astype("category").cat.codes
combined_rolling["hour"] = combined_rolling["time"].str.replace(":.+", "", regex=True).astype("int")
combined_rolling["date"] = pd.to_datetime(combined_rolling["date"])
combined_rolling["day_code"] = combined_rolling["date"].dt.dayofweek

new_features = ['venue_code', 'opp_code', 'hour', 'day_code']
combined_rolling.shape

(1009, 38)

In [15]:
# Splitting the dataset into training and testing sets
np.random.seed(42)
msk = np.random.rand(len(combined_rolling)) < 0.5
train_df = combined_rolling[msk]
test_df = combined_rolling[~msk]

# Defining the features and target variable
features = ['gf_rolling', 'ga_rolling', 'sh_rolling', 'sot_rolling', 'pk_rolling', 'pkatt_rolling', 'saves_rolling', 'cs_rolling',
            'venue_code', 'opp_code', 'hour', 'day_code']
X_train = train_df[features]
y_train = train_df['target']
X_test = test_df[features]
y_test = test_df['target']

In [16]:
# Training Logistic Regression model
clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True,
                         intercept_scaling=1, class_weight='balanced', random_state=None,
                         solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)
clf.fit(X_train, np.ravel(y_train))

# Making predictions
y_pred = clf.predict(X_test)

# Calculating accuracy and precision
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')

Accuracy: 0.5337301587301587
Precision: 0.4747903297534686


C:\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [17]:
from sklearn.ensemble import RandomForestClassifier

# Training Random Forest model with a fixed random state
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, np.ravel(y_train))

# Making predictions
y_pred_rf = rf.predict(X_test)

# Calculating accuracy and precision for Random Forest
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')

print(f'Random Forest Accuracy: {accuracy_rf}')
print(f'Random Forest Precision: {precision_rf}')

Random Forest Accuracy: 0.5793650793650794
Random Forest Precision: 0.5293528336911038
